In [17]:
import numpy as np
from pulp import LpProblem, LpVariable, LpBinary, lpSum, LpStatus, LpMinimize, LpMaximize, value, LpInteger
import matplotlib.pyplot as plt

In [18]:
# get price and run time for each functions in chains
f1_time = np.fromfile('f1_duration.dat')
f1_price = np.fromfile('f1_cost.dat')

f2_time = np.fromfile('f2_duration.dat')
f2_price = np.fromfile('f2_cost.dat')

f3_time = np.fromfile('f3_duration.dat')
f3_price = np.fromfile('f3_cost.dat')

f4_time = np.fromfile('f4_duration.dat')
f4_price = np.fromfile('f4_cost.dat')


time_core1 = {1: f1_time, 2:f2_time, 3:f3_time, 4:f4_time}
price_core = {1:f1_price, 2:f2_price, 3:f3_price, 4:f4_price}

# min max memory of the system in MB
max_memory = 2999
min_memory = 128
# alpha is the step in MB for RAM that can be taken
alpha = 1


print (min(f1_time) + min(f2_time)+max(min(f3_time), min(f4_time))) # minimum possible execution time possible 


1346.9081321366843


In [19]:
def getPrice_n_t(p):
    '''
    returns price for the vf_id=p[0] at memory=p[1]
    '''
    memory = p[0]
    n_type = p[1]
    vf_id = p[2]
    if n_type == 'C1':
        r_value = price_core[vf_id][memory-128]
    elif n_type == 'C2':
        r_value = price_core[vf_id][memory-128]
    else:
        r_value = price_edge[vf_id][memory-128]
    return r_value

In [20]:
def getTime_n_t(p):
    '''
    returns running time for the vf_id=p[0] at memory=p[1]
    '''
    memory = p[0]
    n_type = p[1]
    vf_id = p[2]
    if n_type == 'C1':
        r_value = time_core1[vf_id][memory-128]
    elif n_type == 'C2': 
        r_value = time_core2[vf_id][memory-128]
    else:
        r_value = time_edge[vf_id][memory-128]
    return r_value

In [21]:
def ILP_SP_NodeType(SG, T):
    '''
    :param SC: the service chain array with VF ids. e.g. [1,3,2]
    :param T: The thrushold value on the running time of service chain
    :return dict{vf_1: mem_1, vf_2: mem_2, ..., vf_N: mem_N}
    '''
    assignments = {}
    functions = set()
    
    for p in SG:
        functions.update(p)
    functions = list(functions)
    
    # min max memory of the system in MB
    max_memory = 2999
    min_memory = 128
    # alpha is the step in MB for RAM that can be taken
    alpha = 1
    no_of_variables = int((max_memory-min_memory)/alpha)
    
    # get memory options that we have
    mem = []
    for i in range(no_of_variables+1):
        mem.append(128 + i*alpha)
    
    # node type
    node_type = ['C1']
    
    
    for f in functions: 
        assignments[f] = [(m, n, f) for m in mem for n in node_type]
   
    all_assignments = []
    for f in assignments:
        all_assignments+= assignments[f]
    
    
    X_var = LpVariable.dicts("x", all_assignments, 0, 1, LpBinary)
    
    # define the ILP problem
    prob = LpProblem("Service Provider Chain Placement", LpMinimize)

    # objective function
    prob += lpSum(X_var[p]*getPrice_n_t(p) for p in all_assignments)

    # SUBJECT TO:
    
    # only one memory is selected for each element in service chain
    for f in functions: #functions
        prob += lpSum(X_var[(m, n, f)] for m in mem for n in node_type) == 1

    for pt in SG: # [1,2,3]
        path_assignments = []
        for f in pt: 
            path_assignments += assignments[f]#concatinate all the assignments for functions in pt 
        prob += lpSum(X_var[p]*getTime_n_t(p) for p in path_assignments) <= T
        
    
    # solve the ILP
    prob.solve()

    # get dictionary to return
    dict_values ={}
    
    if prob.status == 1:
        for s in functions:
            for n in node_type:
                for m in mem:
                    if X_var[(m, n, s)].varValue > 0:
                        if n == 'C1':
                            dict_values[s] = [m, 'C1']
                            print ('Memory Selected: ', m, '  for service chain:', s, ' on CORE1')
                        else:
                            dict_values[s] = [m, 'E']
                            print ('Memory Selected: ', m, '  for service chain:', s, ' on EDGE')
    else:
        print ("Cannot satisfy request.")
        
    return dict_values

In [28]:
delay = [1346.9071321366843] # replace this with the SLO

SG = [[1, 2, 3], [1,2,4]] # a user has to encode all the unique paths here -- in current case, there are 2 paths only.
solutions = {}
for d in delay:
    s = ILP_SP_NodeType(SG,d)
    solutions[d] = s
    

print(solutions)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/aliraza/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/9l/nb5rkvcx70xgfcttqmzqc7dm0000gn/T/b8e06fe7033e46e799dad7c276063ad3-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/9l/nb5rkvcx70xgfcttqmzqc7dm0000gn/T/b8e06fe7033e46e799dad7c276063ad3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 63196 RHS
At line 63203 BOUNDS
At line 74692 ENDATA
Problem MODEL has 6 rows, 11488 columns and 28720 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.02 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.05   (Wallclock seconds):       0.07

Cannot satisfy request.
